In [1]:
import fastai
from fastai.vision import *

In [2]:
work_dir = Path('/kaggle/working/')
path = Path('../input')

In [3]:
train = 'train_images/train_images'
test =  path/'leaderboard_test_data/leaderboard_test_data'
holdout = path/'leaderboard_holdout_data/leaderboard_holdout_data'
sample_sub = path/'SampleSubmission.csv'
labels = path/'traininglabels.csv'

In [4]:
test_names = [f for f in test.iterdir()]
holdout_names = [f for f in holdout.iterdir()]

In [5]:
df = pd.read_csv(labels)
df_sample = pd.read_csv(sample_sub)

In [6]:
df.head()

,image_id,has_oilpalm,score
0,img_000002017.jpg,0,0.7895
1,img_000012017.jpg,0,1.0000
2,img_000022017.jpg,0,1.0000
3,img_000072017.jpg,0,1.0000
4,img_000082017.jpg,0,1.0000


In [7]:
df = df[df.score>0.85]
len(df)

12724

In [8]:
N = len(df); N

12724

In [9]:
def get_data(size=128, bs=64, pct=0.2, seed=2019):
    src = (ImageItemList.from_df(df, path, folder=train)
           .random_split_by_pct(pct, seed=seed)
           .label_from_df('has_oilpalm')
          .add_test(test_names+holdout_names))
    return (src.transform(get_transforms(flip_vert=True, max_lighting=0.5), size=size)
            .databunch(bs=bs)
            .normalize(imagenet_stats))

In [10]:
#This was working perfectly some minutes ago!
from sklearn.metrics import roc_auc_score
def auc_score(y_score,y_true):
    return torch.tensor(roc_auc_score(y_true,y_score[:,1])) # use as metric

def get_learner(data):
    return create_cnn(data, models.resnet50, 
                       metrics=[accuracy], 
                       model_dir='/kaggle/working/models')

# X-validation

In [11]:
# sample = 500
folds = 2
preds = 0
for i in range(folds):
    print(f'---Fold {i}---')
    data = get_data(256, 16, seed=i)
    learn = get_learner(data)
    learn.freeze()
    learn.fit_one_cycle(12, slice(1e-2))
    learn.save(f'256-{i}')
    learn.unfreeze()
    learn.fit_one_cycle(8, slice(1e-4,1e-3))
    learn.save(f'256u-{i}')
    p,t = learn.get_preds()
    print(f'AUC score after fold {i} :{auc_score(p,t)}')
    p,t = learn.TTA(ds_type=DatasetType.Test)
    preds += to_np(p[:,1])/folds

---Fold 0---


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /tmp/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:01<00:00, 66018290.56it/s]


epoch,train_loss,valid_loss,accuracy
1,0.051124,0.010230,0.996855
2,0.029414,0.012454,0.994104
3,0.060772,0.017993,0.998035


In [12]:
preds.shape

(6534,)

## Sub file

In [13]:
ids = np.array([f.name for f in test_names]+[f.name for f in holdout_names])

In [14]:
ids.shape

(6534,)

In [15]:
sub = pd.DataFrame(np.stack([ids, preds], axis=1), columns=df_sample.columns)

In [16]:
sub.to_csv(work_dir/'sub-k-fold.csv', index=False)